## Deep Neural Network for MNIST Classification

##### Goal is to write an algorithm that detects which digit is handwritten
##### Since there are 10 digits, the classification problem will have 10 classes
##### Our goal will be to build a neural network with 2 hidden layers

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

#### Load the dataset

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# 'as_supervised' Loads the data in a 2-tuple structure (input and target)
# 'with_info' provides a tuple containing info about the version, features and number of samples of the dataset

### Pre-Processing the data

In [3]:
# by default Tensorflow's mnist has training and testing datasets but no validation datasets so we have to split data ourselves

mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# creates validation dataset by the number of training samples divided by 10

num_validation_samples = tf.cast(num_validation_samples, tf.int64)
# tf.cast(x, datatype) converts the variable into a given data type to make sure validation dataset is an intager
# preventing any potential issues caused by floats

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
# stores the number of test samples in a variable

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label
# this function scales the data to make the result more numerically stable (e.g. inputs between 0 and 1)
# on images, the matrix is made of intagers between 0 and 255, so we should convert this to floats between 0 and 1

scaled_train_and_validation_data = mnist_train.map(scale)
# dataset.map(function) applies a custom transformation to a given dataset.
# It takes a function which determines the transformation as input

### Shuffle the data

In [4]:
BUFFER_SIZE = 10000
# shuffling should be >= number of samples to happen uniformly
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# batch size of 1 = Stochastic Gradient Descent (SGD)
# batch size of # of samples = (single batch) GD
# we want a batch size >1 but < # samples = mini-batch GD
# Increasing batch size will increase completion speed but largely decrease accuracy of model
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
test_data = mnist_test.map(scale)


# the model expects the validation dataset to be in batch-form too so we should override validation data with .batch
validation_data = validation_data.batch(num_validation_samples)
# same with the test data
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))
# the function 'iter' creates an object which can be iterated one element at a time in a loop
# 'next' loads the next batch (next element of an iterable object)

### Model - outline the model

In [5]:
input_size = 784
output_size = 10
hidden_layer_size = 200

# the keras.sequential function is laying down the model used to stack layers
# the keras.layers.Flatten function transforms a tensor into a vector
# the keras.layers.Dense function takes inputs and finds the Dot product of inputs and weights and adds the bias
# we also apply the activation function here
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
    tf.keras.layers.Dense(output_size, activation='softmax'),
])

### Choose the optimizer and Function Loss

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# categorical_crossentropy expects that you've one-hot encoded the target data already
# sparse_categorical_crossentropy applies the one-hot encoding
# finally, the 3rd argument includes the 'metrics' we want to process throughout the training and testing processes
# (in most cases, this is accuracy)

### Training

In [7]:
num_epochs = 5
# provide the model with the set number of epochs, validation inputs and targets,
# set verbose to 2 to only include important info from each epoch
model.fit(train_data, epochs = num_epochs, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 6s - loss: 0.5251 - accuracy: 0.8499 - val_loss: 0.1740 - val_accuracy: 0.9537
Epoch 2/5
540/540 - 4s - loss: 0.1467 - accuracy: 0.9584 - val_loss: 0.1037 - val_accuracy: 0.9693
Epoch 3/5
540/540 - 4s - loss: 0.0960 - accuracy: 0.9720 - val_loss: 0.0871 - val_accuracy: 0.9750
Epoch 4/5
540/540 - 4s - loss: 0.0736 - accuracy: 0.9785 - val_loss: 0.0672 - val_accuracy: 0.9815
Epoch 5/5
540/540 - 4s - loss: 0.0572 - accuracy: 0.9828 - val_loss: 0.0644 - val_accuracy: 0.9822


In [8]:
# the training accuracy is just the average accuracy across batches whereas validation accuracy is of the whole validation set
# the validation accuracy at the end of the last epoch is the True accuracy of the validation dataset.
# almost there but not fully tested the accuracy of the model yet.
# the final accuracy of the model comes from the forward propagation of the test dataset, not the validation one.
# this is because we may be overfitting the validation dataset by fiddling with the hyperparameters
# the test dataset makes sure our hyperparameters (width, depth, batch size, # epochs etc) don't overfit!

### Test the Model

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 715ms/step - loss: 0.1057 - accuracy: 0.9693


In [10]:
# lets format it to make it appear neater and easier to read
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.11. Test accuracy: 96.93%


## important!
#### after running the final test for the model, we're no longer allowed to change it as...
#### ... if it is changed it will no longer be a dataset the model has never seen (as it has seen it)

In [11]:
# Getting a test accuracy very close to the validation accuracy shows that we have not overfit!
# this accuracy is the accuracy expected to get if we test the model in the real world.